In [1]:
import socket

In [2]:
# import adafruit_ble
# from adafruit_ble import BLERadio

In [8]:
radio = BLERadio()
print("Scanning...")
found = set()
for entry in radio.start_scan(timeout=60, minimum_rssi=-80):
    addr = entry.address
    print(addr)
    if addr not in found:
        print(entry)
    found.add(addr)

Scanning...


In [ ]:
from bleak import BleakScanner

async def run():
    devices = await BleakScanner.discover()
    for d in services:
        print(d)

loop = asyncio.get_event_loop()
loop.run_until_complete(run())

In [ ]:
from bleak import BleakClient
async with BleakClient(address) as client:
    res = client.read_gatt_char()

### PyBluez

In [9]:
import bluetooth
devices = bluetooth.discover_devices(duration=10)
devices

['64:B5:C6:41:59:E0', '1C:15:1F:C4:05:47', '34:7D:F6:0A:09:AF']

In [2]:
address = '64:B5:C6:41:59:E0'
puerto = 2445

In [3]:
import asyncio

In [5]:
devices

['00:0B:CE:01:98:9A',
 '64:B5:C6:41:59:E0',
 'AC:C1:EE:00:DA:3F',
 '24:4B:03:34:BF:5F',
 'CF:C3:31:28:2B:EC']

In [25]:
address = devices[2]
address

'AC:C1:EE:00:DA:3F'

In [26]:
if not bluetooth.is_valid_address(address):
    print("Dirección", address, "no válida.")
    
sock = bluetooth.BluetoothSocket( bluetooth.RFCOMM )
sock.bind((address, 1))
sock.listen(1)

OSError: Intento de acceso a un socket no permitido por sus permisos de acceso.


In [46]:
devices

['00:0B:CE:01:98:9A',
 '64:B5:C6:41:59:E0',
 'AC:C1:EE:00:DA:3F',
 '24:4B:03:34:BF:5F',
 'CF:C3:31:28:2B:EC']

In [32]:
bluetooth.find_service(address= address)

[{'host': 'AC:C1:EE:00:DA:3F',
  'name': None,
  'description': '',
  'port': 31,
  'protocol': 'L2CAP',
  'rawrecord': b'6\x000\t\x00\x00\n\x00\x01\x00\x00\t\x00\x015\x03\x19\x18\x01\t\x00\x045\x135\x06\x19\x01\x00\t\x00\x1f5\t\x19\x00\x07\t\x00\x01\t\x00\x05\t\x00\x055\x03\x19\x10\x02',
  'service-classes': [b'1801'],
  'profiles': [],
  'provider': None,
  'service-id': None,
  'handle': 65536},
 {'host': 'AC:C1:EE:00:DA:3F',
  'name': None,
  'description': '',
  'port': 31,
  'protocol': 'L2CAP',
  'rawrecord': b'6\x000\t\x00\x00\n\x00\x01\x00\x01\t\x00\x015\x03\x19\x18\x00\t\x00\x045\x135\x06\x19\x01\x00\t\x00\x1f5\t\x19\x00\x07\t\x00\x14\t\x00\x1c\t\x00\x055\x03\x19\x10\x02',
  'service-classes': [b'1800'],
  'profiles': [],
  'provider': None,
  'service-id': None,
  'handle': 65537},
 {'host': 'AC:C1:EE:00:DA:3F',
  'name': b'Headset Gateway\x00',
  'description': '',
  'port': 2,
  'protocol': 'RFCOMM',
  'rawrecord': b'6\x00N\t\x00\x00\n\x00\x01\x00\x03\t\x00\x015\x06\x19\x1

In [31]:
bluetooth.find_service()

[{'host': '24:4B:03:34:BF:5F',
  'name': None,
  'description': '',
  'port': 31,
  'protocol': 'L2CAP',
  'rawrecord': b'6\x000\t\x00\x00\n\x00\x01\x00\x00\t\x00\x015\x03\x19\x18\x01\t\x00\x045\x135\x06\x19\x01\x00\t\x00\x1f5\t\x19\x00\x07\t\x00\x01\t\x00\x05\t\x00\x055\x03\x19\x10\x02',
  'service-classes': [b'1801'],
  'profiles': [],
  'provider': None,
  'service-id': None,
  'handle': 65536},
 {'host': '24:4B:03:34:BF:5F',
  'name': None,
  'description': '',
  'port': 31,
  'protocol': 'L2CAP',
  'rawrecord': b'6\x000\t\x00\x00\n\x00\x01\x00\x01\t\x00\x015\x03\x19\x18\x00\t\x00\x045\x135\x06\x19\x01\x00\t\x00\x1f5\t\x19\x00\x07\t\x00\x14\t\x00\x1e\t\x00\x055\x03\x19\x10\x02',
  'service-classes': [b'1800'],
  'profiles': [],
  'provider': None,
  'service-id': None,
  'handle': 65537},
 {'host': '24:4B:03:34:BF:5F',
  'name': b'Samsung Smart TV Audio\x00',
  'description': '',
  'port': 23,
  'protocol': 'L2CAP',
  'rawrecord': b'6\x00\x8c\t\x00\x00\n\x00\x01\x00\x03\t\x00\x015\

In [32]:
bluetooth.find_service(address= address)

[{'host': 'AC:C1:EE:00:DA:3F',
  'name': None,
  'description': '',
  'port': 31,
  'protocol': 'L2CAP',
  'rawrecord': b'6\x000\t\x00\x00\n\x00\x01\x00\x00\t\x00\x015\x03\x19\x18\x01\t\x00\x045\x135\x06\x19\x01\x00\t\x00\x1f5\t\x19\x00\x07\t\x00\x01\t\x00\x05\t\x00\x055\x03\x19\x10\x02',
  'service-classes': [b'1801'],
  'profiles': [],
  'provider': None,
  'service-id': None,
  'handle': 65536},
 {'host': 'AC:C1:EE:00:DA:3F',
  'name': None,
  'description': '',
  'port': 31,
  'protocol': 'L2CAP',
  'rawrecord': b'6\x000\t\x00\x00\n\x00\x01\x00\x01\t\x00\x015\x03\x19\x18\x00\t\x00\x045\x135\x06\x19\x01\x00\t\x00\x1f5\t\x19\x00\x07\t\x00\x14\t\x00\x1c\t\x00\x055\x03\x19\x10\x02',
  'service-classes': [b'1800'],
  'profiles': [],
  'provider': None,
  'service-id': None,
  'handle': 65537},
 {'host': 'AC:C1:EE:00:DA:3F',
  'name': b'Headset Gateway\x00',
  'description': '',
  'port': 2,
  'protocol': 'RFCOMM',
  'rawrecord': b'6\x00N\t\x00\x00\n\x00\x01\x00\x03\t\x00\x015\x06\x19\x1

In [36]:
host = 'AC:C1:EE:00:DA:3F'
name = b'Advanced Audio\x00'
port = 25

In [45]:
socket = bluetooth.BluetoothSocket( bluetooth.RFCOMM )
socket.connect((host, port))

OSError: 

In [27]:
if not bluetooth.is_valid_address(address):
    print("Dirección", address, "no válida.")
    
sock = bluetooth.BluetoothSocket( bluetooth.RFCOMM )
sock.bind((address, 1))
sock.listen(1)

OSError: Intento de acceso a un socket no permitido por sus permisos de acceso.


In [ ]:
# bluetooth.